# Notebook 2: finding the transcript lengths

author: jake siegel

date: 11.29.2017

The purpose of this notebook is to figure out how to calculate the transcript length for the RNA Seq data using the .gtf file downloaded for gencode v17 from gencode.  The file is in /data/raw

The gtf is massive... it's 1.03GB.  This may be too big to read in all at once.  It might have to be read in in chunks.

The v17.annotation file contains fewer genes than the RNA-Seq processed dataset (~57,400).

In [1]:
%matplotlib inline
import pandas as pd
import time

# data_src = '../data/raw/gencode.v17.annotation.gtf'
data_src = '../data/raw/gencode.v17.chr_patch_hapl_scaff.annotation.gtf'
df = pd.read_csv(data_src, skiprows=5, header=None, usecols=[1,2,3,4,8],sep = '\t')
print df.shape
df.head()

(2823730, 5)


,1,2,3,4,8
0,HAVANA,gene,11869,14412,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
1,HAVANA,transcript,11869,14409,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
2,HAVANA,exon,11869,12227,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
3,HAVANA,exon,12613,12721,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
4,HAVANA,exon,13221,14409,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."


In [ ]:
# How many genes are there in this build?
df[8].astype(str).str.split('"', expand=True)[1].unique().shape

In [ ]:
# Can I just use the Ensembl genes? No.
test = df.loc[df[1]=="ENSEMBL"]
test[8].astype(str).str.split('"', expand=True)[1].unique().shape

In [2]:
df['gene_id']=df[8].astype(str).str.split('"', expand=True)[1]
df['transcript_id']=df[8].astype(str).str.split('"', expand=True)[3]
df.head()

,1,2,3,4,8,gene_id,transcript_id
0,HAVANA,gene,11869,14412,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENSG00000223972.4
1,HAVANA,transcript,11869,14409,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2
2,HAVANA,exon,11869,12227,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2
3,HAVANA,exon,12613,12721,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2
4,HAVANA,exon,13221,14409,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2


In [3]:
df['bp']=df[4]-df[3]+1
df.head()

,1,2,3,4,8,gene_id,transcript_id,bp
0,HAVANA,gene,11869,14412,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENSG00000223972.4,2544
1,HAVANA,transcript,11869,14409,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2,2541
2,HAVANA,exon,11869,12227,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2,359
3,HAVANA,exon,12613,12721,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2,109
4,HAVANA,exon,13221,14409,"gene_id ""ENSG00000223972.4""; transcript_id ""EN...",ENSG00000223972.4,ENST00000456328.2,1189


OK, so here is where we need to figure out how to calculate the "right" transcript length for each transcript.  

The library prep protocol in the paper is a poly-A-selective protocol, so we need the length of the mature transcript.  That means summing over exons.  Many of the genes in this annotation have multiple entries (for example, see below).  

So, my plan is to sum over exons for each transcript_id and then take the maximum (by length) of all of the mature transcript_id's for a given gene_id as the length of that gene.

In [ ]:
df.iloc[:20,:]

In [7]:
# There is a row that is annotated 'gene' in column 2 that is unique for each gene_id.
# That makes for a convenient hierarchy for keeping track of the calculated transcript lengths.
# I am going to put the calculted mature transcript length for each transcript in the row annoted as "transcript"
# in column 2.
# Then I am going to put the maximum calculated transcript length for a given gene in the row annoted "gene".
print 'number of gene_id:', df['gene_id'].loc[df[2]=='gene'].unique().shape
print 'number of transcript_id:', df['transcript_id'].loc[df[2]=='transcript'].unique().shape

number of gene_id: (62798,)
number of transcript_id: (211826,)


In [8]:
# Here is a way to do the summation for a given transcript:
import time

start = time.time()
temp = df['bp'].loc[(df['transcript_id']=='ENST00000515242.2')&(df[2]=='exon')].sum()
end = time.time()
print 'summing over 1 transcript id took:', (end-start),'seconds'

summing over 1 transcript id took: 3.26815080643 seconds


Yeah, that's going to take 8 days to do it that way.  Need to think of a different option.

In [23]:
# Here we loop over all unique transcripts and put the calculated length in the row annotated "transcript" in column 2.
df2 = pd.DataFrame()
df2 = df.iloc[:20,:]
df2.shape
start = time.time()
df2.groupby(['gene_id','transcript_id',2])['bp'].sum()
end = time.time()
print 'groupby summation for 4 transcipt ids takes:',(end-start),'seconds'


# for transcript in df['transcript_id'].loc[df[2]!='gene'].unique():
#     df['bp'].loc[(df['transcript_id']==transcript)&(df[2]=='transcript')]=df['bp'].loc[(df['transcript_id']==transcript)&(df[2]=='exon')].sum()

# df.iloc[:20,:]

groupby summation for 4 transcipt ids takes: 0.00754117965698 seconds


In [40]:
lengths = pd.DataFrame()
lengths = df.groupby(['gene_id','transcript_id',2])['bp'].sum()
lengths = lengths.reset_index()
lengths.head()

,gene_id,transcript_id,2,bp
0,ENSG00000000003.10,ENSG00000000003.10,gene,11322
1,ENSG00000000003.10,ENST00000373020.4,CDS,735
2,ENSG00000000003.10,ENST00000373020.4,UTR,1471
3,ENSG00000000003.10,ENST00000373020.4,exon,2206
4,ENSG00000000003.10,ENST00000373020.4,start_codon,3


Now we just need to select the maximum exon length for each gene_id

In [43]:
transcript_lengths = lengths.loc[lengths[2]=='exon'].groupby('gene_id')['bp'].max().reset_index()
transcript_lengths.head()

,gene_id,bp
0,ENSG00000000003.10,2206
1,ENSG00000000005.5,1339
2,ENSG00000000419.8,1161
3,ENSG00000000457.8,3477
4,ENSG00000000460.12,4355


In [44]:
transcript_lengths.shape

(62798, 2)

In [45]:
transcript_lengths.to_csv('../data/interim/transcipt_lengths.csv', encoding='utf-8', index=False)

# Final Remarks

It seems this did the trick.  This version of the GENCODE v17 annotation has gene_ids that match the read count data and is contains more genes than the read count file, so hopefully every read gene has a transcript length.

The processed lengths are in the /data/interim folder.

The next step is to go back to the read count data and normalize it to TPM. I probably need to figure out what those spike-in control reads mean.  Bare minimum, I should dump them.  I wonder, though, if people use them as a way to normalize the reads.  That seems silly, but there might be some value in that data that I am overlooking.

In [1]:
%matplotlib inline
import pandas as pd
import time

# data_src = '../data/raw/gencode.v17.annotation.gtf'
data_src = '../data/raw/gencode.v17.chr_patch_hapl_scaff.annotation.gtf'
df = pd.read_csv(data_src, skiprows=5, header=None,sep = '\t')
print df.shape
df.head()

(2823730, 9)


,0,1,2,3,4,5,6,7,8
0,chr1,HAVANA,gene,11869,14412,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
1,chr1,HAVANA,transcript,11869,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
2,chr1,HAVANA,exon,11869,12227,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
3,chr1,HAVANA,exon,12613,12721,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."
4,chr1,HAVANA,exon,13221,14409,.,+,.,"gene_id ""ENSG00000223972.4""; transcript_id ""EN..."


In [3]:
df[8].astype(str).str.split('"', expand=True)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,gene_id,ENSG00000223972.4,; transcript_id,ENSG00000223972.4,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
1,gene_id,ENSG00000223972.4,; transcript_id,ENST00000456328.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
2,gene_id,ENSG00000223972.4,; transcript_id,ENST00000456328.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
3,gene_id,ENSG00000223972.4,; transcript_id,ENST00000456328.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
4,gene_id,ENSG00000223972.4,; transcript_id,ENST00000456328.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
5,gene_id,ENSG00000223972.4,; transcript_id,ENST00000515242.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
6,gene_id,ENSG00000223972.4,; transcript_id,ENST00000515242.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
7,gene_id,ENSG00000223972.4,; transcript_id,ENST00000515242.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
8,gene_id,ENSG00000223972.4,; transcript_id,ENST00000515242.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
9,gene_id,ENSG00000223972.4,; transcript_id,ENST00000518655.2,; gene_type,pseudogene,; gene_status,KNOWN,; gene_name,DDX11L1,...,None,None,None,None,None,None,None,None,None,None
